In [12]:
import torch
import os
import time
import cv2
import mlflow

import pandas as pd
import numpy as np
import torchvision.transforms as ttinter
import dl_ops

from torchvision.transforms import v2
from torch.utils.data.dataloader import DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
from torchinfo import summary

try:
    from emotions_model_training.ERNN import ERNN
    from emotions_model_training.dl_ops import dl_ops
    from SQL.connector import connector
    from emotions_model_training.EmotionsDataset import EmotionsDataset
    from emotions_model_training.add_files.ResBlock import ResBlock
    from emotions_model_training.add_files.torch_transforms import torch_transforms
except ModuleNotFoundError:
    os.chdir("..")
    from emotions_model_training.ERNN import ERNN
    from emotions_model_training.dl_ops import dl_ops
    from SQL.connector import connector
    from emotions_model_training.EmotionsDataset import EmotionsDataset
    from emotions_model_training.add_files.ResBlock import ResBlock
    from emotions_model_training.add_files.torch_transforms import torch_transforms

In [13]:
# Checking CUDA avalibility
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print("Using " + str(device))


Using cuda


In [14]:
# Paramteres for scripts (implement proportion of sets)
params = {
    "sets_proportion": [0.6, 0.2, 0.2],       # train / valid / test
    "batch": 256,                              # batch size
    "do_shuffle": True,                       # shuffling data in loader
    "learn_rate": 1e-3,                       # learning rate for training
    "num_epoch": 10,                           # training epochs
    "is_padding": False,                      # True - padding, False - resizing
    "Res_Block_num": [3, 3, 6, 3]             # Config for Res_Block (num of layers)
}

# Description of model for MLFlow tagging
model_info = "ERNN_03 (ResLike) with SGD and higher batch"
model_name = "ERNN_ResLike_03"

# Defining model and parameters
# Currently if you want to make changes to ResBlock - do it inside ERNN Module
ernn = ERNN(ResBlock, params["Res_Block_num"]).to(device)

# MOD_PATH = 'D:\\Repos\\Face_Detection_PyTorch\\emotions_model_training\\add_files\\model_2.pt'
# ernn.load_state_dict(torch.load(MOD_PATH))

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(ernn.parameters(), lr=params["learn_rate"])

params["loss_fn"] = str(loss_fn)
params["optimizer"] = str(optim)

In [15]:
# Loading the dataset with SQL query and dispaying for insight
db_dir = "D:\\Repos\\Face_Detection_PyTorch\\dataset\\expression_dataset\\data\\data\\image\\origin\\"

conn = connector()
db = conn.query(f'SELECT * FROM dbo.label')

display(db)


d:\Repos\Face_Detection_PyTorch\SQL\connector.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)


,image,face_id_image,box_top,box_left,box_right,box_bottom,box_confidence,label
0,distaste_mother_319.jpg,3,120,664,712,168,33.1615,6
1,distaste_mother_498.jpg,1,175,103,150,222,7.75454,3
2,distaste_mother_58.jpg,0,72,88,136,120,72.9418,6
3,distaste_mother_623.jpg,0,254,5,72,321,79.3727,6
4,distaste_mother_667.jpg,0,67,180,315,202,61.9211,1
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [16]:
# Splitting data into sets (train, test, valid) - with renumeration
train, val_test = train_test_split(db, test_size=0.4, train_size=0.6, random_state=42, stratify=db['label'])
valid, test = train_test_split(val_test, test_size=0.5, train_size=0.5, random_state=42, stratify=val_test['label'])

train, valid, test = train.reset_index(drop=True), valid.reset_index(drop=True), test.reset_index(drop=True)

sets = [train, valid, test]
sets_files = ["train.csv", "valid.csv", "test.csv"]
for iter, set in enumerate(sets_files):
    sets[iter].to_csv(f"emotions_model_training\\add_files\\sets_save\\{set}")

# If you want to continue training on the saved model with the same set splits

# SETS_PATH = "emotions_model_training\\add_files\\sets_save\\"
# train, valid, test = pd.read_csv(SETS_PATH + "train.csv"), pd.read_csv(SETS_PATH + "valid.csv"), pd.read_csv(SETS_PATH + "test.csv")

print("Checking sizes of prepared datasets:")
print("Train shape: " + str(train.shape[0]) + " (" + str(round((train.shape[0]/db.shape[0])*100)) + "%)")
print("Valid shape: " + str(valid.shape[0]) + " (" + str(round((valid.shape[0]/db.shape[0])*100)) + "%)")
print("Test shape: " + str(test.shape[0]) + " (" + str(round((test.shape[0]/db.shape[0])*100)) + "%)")

Checking sizes of prepared datasets:
Train shape: 55075 (60%)
Valid shape: 18359 (20%)
Test shape: 18359 (20%)


In [17]:
# Mean and std calculated in detect_analysis.ipynb for each channel
mean = [0.542, 0.425, 0.374]
std = [0.231, 0.208, 0.198]

# Init for datasets with PyTorch class
train_dataset = EmotionsDataset(train, db_dir, mean=mean, std=std)

# Fixing valid and test dataset to same sizes (for convinience with NN architecture)
horz, vert = train_dataset.horz_max, train_dataset.vert_max

# Init for datasets
valid_dataset = EmotionsDataset(valid, db_dir, mean=mean, std=std, horz_max=horz, vert_max=vert, padding=params["is_padding"])
test_dataset = EmotionsDataset(test, db_dir, mean=mean, std=std, horz_max=horz, vert_max=vert, padding=params["is_padding"])

# Init for dataloaders with default batch = 64
train_dataloader = DataLoader(dataset=train_dataset, batch_size=params["batch"], shuffle=params["do_shuffle"], drop_last=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=params["batch"], shuffle=params["do_shuffle"], drop_last=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=params["batch"], shuffle=params["do_shuffle"], drop_last=True)


d:\Repos\Face_Detection_PyTorch\face_venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [18]:
# Checking maximal size - in theory it can vary, but it depends from split of dataset
print("Maximal size of images: " + str(horz) + " x " + str(vert))

Maximal size of images: 2117 x 2117


In [19]:
# Creating operational class for proceding with train, valid, test process
dl_ops = dl_ops(model=ernn, loss_fn=loss_fn, optimizer=optim)
print(f"Architecture of current ERNN: \n {summary(ernn)}")

Architecture of current ERNN: 
Layer (type:depth-idx)                   Param #
ERNN                                     --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       9,472
│    └─BatchNorm2d: 2-2                  128
│    └─ReLU: 2-3                         --
├─MaxPool2d: 1-2                         --
├─Sequential: 1-3                        --
│    └─ResBlock: 2-4                     --
│    │    └─Sequential: 3-1              37,056
│    │    └─Sequential: 3-2              37,056
│    │    └─ReLU: 3-3                    --
│    └─ResBlock: 2-5                     --
│    │    └─Sequential: 3-4              37,056
│    │    └─Sequential: 3-5              37,056
│    │    └─ReLU: 3-6                    --
│    └─ResBlock: 2-6                     --
│    │    └─Sequential: 3-7              37,056
│    │    └─Sequential: 3-8              37,056
│    │    └─ReLU: 3-9                    --
├─Sequential: 1-4                        --
│    └─ResBl

In [20]:
with open("emotions_model_training\\add_files\\model_arch.txt", "w", encoding='utf-8') as f:
    f.write(str(summary(ernn)))
f.close()

In [21]:
# Parameters summary
iter = 1
print("Parameters:")
for key, item in params.items():
    print(f"{iter}. {key} --> {item}")
    iter += 1

Parameters:
1. sets_proportion --> [0.6, 0.2, 0.2]
2. batch --> 128
3. do_shuffle --> True
4. learn_rate --> 0.005
5. num_epoch --> 10
6. is_padding --> False
7. Res_Block_num --> [3, 3, 6, 3]
8. loss_fn --> CrossEntropyLoss()
9. optimizer --> Adagrad (
Parameter Group 0
    differentiable: False
    eps: 1e-10
    foreach: None
    initial_accumulator_value: 0
    lr: 0.005
    lr_decay: 0
    maximize: False
    weight_decay: 0
)


In [22]:
# Model path
PATH = 'D:\\Repos\\Face_Detection_PyTorch\\emotions_model_training\\add_files\\model'

# Setting URI for MLFlow Tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:7000")

# Setting exp name - change possible from main Jupyter notebook
mlflow.set_experiment(model_name)

model_iter = 3
# MFlow training/validation run
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_artifact(local_path="D:\\Repos\\Face_Detection_PyTorch\\emotions_model_training\\add_files\\model_arch.txt")
    mlflow.set_tag("Model Info", model_info)
    for iter in range (params["num_epoch"]):
        print(f"Epoch {iter + 1}:")
        dl_ops.train(dataloader=train_dataloader, device=device)
        mlflow.pytorch.log_model(artifact_path="mlruns", pip_requirements="D:\\Repos\\Face_Detection_PyTorch\\requirements.txt", registered_model_name=model_name, pytorch_model=ernn)
        torch.save(ernn.state_dict(), PATH + f"_{model_iter}.pt")
        model_iter += 1
        dl_ops.valid(dataloader=valid_dataloader, device=device, epoch=iter)

Epoch 1:


d:\Repos\Face_Detection_PyTorch\face_venv\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Batch: 1 (0.002324103495233772) 
 Loss: 1.553296
Batch: 2 (0.004648206990467544) 
 Loss: 1.849224
Batch: 3 (0.006972310485701317) 
 Loss: 1.797884
Batch: 4 (0.009296413980935088) 
 Loss: 1.798760
Batch: 5 (0.01162051747616886) 
 Loss: 1.649797
Batch: 6 (0.013944620971402633) 
 Loss: 1.759172
Batch: 7 (0.016268724466636405) 
 Loss: 1.829484
Batch: 8 (0.018592827961870177) 
 Loss: 1.798234
Batch: 9 (0.020916931457103948) 
 Loss: 1.673234
Batch: 10 (0.02324103495233772) 
 Loss: 1.774796
Batch: 11 (0.025565138447571495) 
 Loss: 1.766984
Batch: 12 (0.027889241942805267) 
 Loss: 1.813859
Batch: 13 (0.030213345438039038) 
 Loss: 1.681047
Batch: 14 (0.03253744893327281) 
 Loss: 1.751359
Batch: 15 (0.03486155242850658) 
 Loss: 1.766984
Batch: 16 (0.03718565592374035) 
 Loss: 1.743547
Batch: 17 (0.039509759418974125) 
 Loss: 1.790422
Batch: 18 (0.041833862914207896) 
 Loss: 1.782609
Batch: 19 (0.04415796640944167) 
 Loss: 1.806047
Batch: 20 (0.04648206990467544) 
 Loss: 1.813859
Batch: 21 (0.048

KeyboardInterrupt: 

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in ernn.state_dict():
    print(param_tensor, "\t", ernn.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optim.state_dict():
    print(var_name, "\t", optim.state_dict()[var_name])

Model's state_dict:
conv1.0.weight 	 torch.Size([64, 3, 7, 7])
conv1.0.bias 	 torch.Size([64])
conv1.1.weight 	 torch.Size([64])
conv1.1.bias 	 torch.Size([64])
conv1.1.running_mean 	 torch.Size([64])
conv1.1.running_var 	 torch.Size([64])
conv1.1.num_batches_tracked 	 torch.Size([])
resblock1.0.conv1.0.weight 	 torch.Size([64, 64, 3, 3])
resblock1.0.conv1.0.bias 	 torch.Size([64])
resblock1.0.conv1.1.weight 	 torch.Size([64])
resblock1.0.conv1.1.bias 	 torch.Size([64])
resblock1.0.conv1.1.running_mean 	 torch.Size([64])
resblock1.0.conv1.1.running_var 	 torch.Size([64])
resblock1.0.conv1.1.num_batches_tracked 	 torch.Size([])
resblock1.0.conv2.0.weight 	 torch.Size([64, 64, 3, 3])
resblock1.0.conv2.0.bias 	 torch.Size([64])
resblock1.0.conv2.1.weight 	 torch.Size([64])
resblock1.0.conv2.1.bias 	 torch.Size([64])
resblock1.0.conv2.1.running_mean 	 torch.Size([64])
resblock1.0.conv2.1.running_var 	 torch.Size([64])
resblock1.0.conv2.1.num_batches_tracked 	 torch.Size([])
resblock1.1.con

In [ ]:
# PATH = 'D:\\Repos\\Face_Detection_PyTorch\\emotions_model_training\\add_files\\model.pt'
# ernn.load_state_dict(torch.load(PATH))
# for iter in range (num_epoch):
#     print(f"Epoch {iter + 1}:")
#     dl_ops.valid(dataloader=valid_dataloader, device=device)

Epoch 1:


d:\Repos\Face_Detection_PyTorch\face_venv\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


UnboundLocalError: cannot access local variable 'accuracy' where it is not associated with a value